In [1]:
library(FlashRLearn)
fm.set.conf("/FlashX/EC2/run_matrix.txt")

Loading required package: pcg
Loading required package: FlashR
Loading required package: RSpectra
Loading required package: Rcpp

Attaching package: 'FlashR'

The following objects are masked from 'package:base':

    cbind, pmax, pmin, rbind



# Measure the performance of FlashR in memory.

In [2]:
data <- fm.runif.matrix(325000000, 40, min = 0, max=1000)
data <- as.integer(data)
data <- fm.materialize(data)

In [3]:
y <- fm.materialize(fm.runif(325000000) > 0.5)

In [4]:
fm.print.mat.info(data)
fm.print.mat.info(y)

In [5]:
fm.set.test.na(FALSE)

NULL

In [6]:
#for (i in 1:5) {
    start <- Sys.time()
    res <- cor(data)
    end <- Sys.time()
    print(end - start)
#}

Time difference of 10.777 secs


In [7]:
#for (i in 1:5) {
    start <- Sys.time()
    res <- fm.svd(data, nu=0, nv=ncol(data))
    end <- Sys.time()
    print(end - start)
#}

Time difference of 7.960912 secs


In [8]:
start <- Sys.time()
res <- naiveBayes.train(data, y)
end <- Sys.time()
print(end - start)

Time difference of 5.226341 secs


In [9]:
data1 <- fm.runif.matrix(336000000, 32, min = 0, max=1000)

In [10]:
start <- Sys.time()
res <- fm.kmeans(data1, 10, iter.max = 5, use.blas = TRUE)
end <- Sys.time()
print(end - start)

KMeans takes 5 iterations and 41.1684 seconds
Time difference of 49.91022 secs


In [19]:
esti.cov.full <- function(X, resp, nk, means, reg.covar)
{
	k <- nrow(means)
	d <- ncol(means)
	covs <- list()
	for (i in 1:k) {
		diff <- sweep(X, 2, means[i,], "-")
		covs[[i]] <- t(resp[,i] * diff) %*% diff / nk[i]
	}
	lapply(covs, function(x) as.matrix(x) + diag(rep(reg.covar, d)))
}

# In this case, we assume all Gaussian distribution has the same covariance matrix.
esti.cov.tied <- function(X, resp, nk, means, reg.covar)
{
	avg.X2 <- t(X) %*% X
	avg.means2 <- t(means * nk) %*% means
	as.matrix((avg.X2 - avg.means2) / sum(nk)) + diag(rep(reg.covar, d))
}

esti.cov.diag <- function(X, resp, nk, means, reg.covar)
{
	avg.X2 <- t(resp) %*% (X * X) / nk
	avg.means2 <- means^2
	avg.Xmeans <- means * (t(resp) %*% X) / nk
	covars <- as.matrix(avg.X2 - 2*avg.Xmeans + avg.means2)
	ifelse(covars < reg.covar, reg.covar, covars)
}

esti.cov.spherical <- function(X, resp, nk, means, reg.covar)
{
	rowMeans(esti.cov.diag(X, resp, nk, means, reg.covar))
}

# This estimate the parameters of Mixture of Gaussian
esti.gaussian.params <- function(X, resp, reg.covar, cov.type)
{
	n <- nrow(X)
	nk <- colSums(resp)
	# a k x d matrix. Each row is the mean of a component.
	means <- t(resp) %*% X / nk
	# a list of covariance matrices for all components.
	covs <- if (cov.type == "full") esti.cov.full(X, resp, nk, means, reg.covar)
		else if (cov.type == "tied") esti.cov.tied(X, resp, nk, means, reg.covar)
		else if (cov.type == "diag") esti.cov.diag(X, resp, nk, means, reg.covar)
		else if (cov.type == "spherical") esti.cov.spherical(X, resp, nk, means, reg.covar)
		else NULL
	list(weights=as.vector(nk)/n, means=as.matrix(means), covs=covs)
}

init.params <- function(X, k, reg.covar, method, cov.type)
{
	N <- nrow(X)
	if (method == "kmeans") {
		res <- fm.kmeans(X, k, 100)
		resp <- fm.matrix(0, nrow(X), k)
		idx <- cbind(fm.seq.int(1, nrow(X), 1), res$cluster)
		resp[idx] <- 1
		# Estimate weights, means and covariances
		params <- esti.gaussian.params(X, resp, reg.covar, cov.type)
		list(weights=params$weights, means=params$means, covs=params$covs)
	}
	else if (method == "random") {
		resp <- fm.runif.matrix(N, k, in.mem=fm.in.mem(X))
		# each row needs to sum up to 1.
		resp <- resp / rowSums(resp)
		# Estimate weights, means and covariances
		params <- esti.gaussian.params(X, resp, reg.covar, cov.type)
		list(weights=params$weights, means=params$means, covs=params$covs)
	}
	else if (method == "random_params") {
		m <- dim(X)[1]
		rand.k <- floor(runif(k, 1, m))
		mus <- X[rand.k,]
		init.covar <- cov(X)
		covars <- list()
		for (i in 1:k)
			covars[[i]] <- init.covar
		phi <- rep.int(1/m, k)
		list(weights=phi, means=mus, covs=covars)
	}
	else
		stop("unknown init method")
}

est.logprob <- function(X, means, covars, cov.type)
{
	n <- nrow(X)
	d <- ncol(X)
	k <- nrow(means)

	comp.logprob.vec <- function(X, mu, covar.vec) {
		X1 <- sweep(X, 2, mu, "-")
		X2 <- sweep(X1, 2, covar.vec, "/")
		rowSums(X2 * X1)
	}

	if (cov.type == "full") {
		logprob <- list()
		for (i in 1:k)
			logprob[[i]] <- fm.dmvnorm(X, means[i,], covars[[i]], log=TRUE)
		return(do.call(cbind, logprob))
	}
	else if (cov.type == "tied") {
		logprob <- list()
		for (i in 1:k)
			logprob[[i]] <- fm.dmvnorm(X, means[i,], covars, log=TRUE)
		return(do.call(cbind, logprob))
	}
	else if (cov.type == "diag") {
		logprob <- list()
		for (i in 1:k) {
			logprob[[i]] <- comp.logprob.vec(X, means[i,], covars[i,])
			logprob[[i]] <- -sum(log(covars[i,]))/2 - 0.5 * (d * log(2 * pi) + logprob[[i]])
		}
		return(do.call(cbind, logprob))
	}
	else if (cov.type == "spherical") {
		logprob <- list()
		for (i in 1:k) {
			logprob[[i]] <- comp.logprob.vec(X, means[i,], rep(covars[i], d))
			logprob[[i]] <- -log(covars[i])*d/2 - 0.5 * (d * log(2 * pi) + logprob[[i]])
		}
		return(do.call(cbind, logprob))
	}
}

est.weighted.logprob <- function(X, means, covars, cov.type, weights)
{
	ret <- est.logprob(X, means, covars, cov.type)
#	ret <- fm.materialize(est.logprob(X, means, covars, cov.type))
	sweep(ret, 2, log(weights), "+")
}

logsumexp <- function(X)
{
	max.X <- fm.agg.mat(X, 1, fm.bo.max)
	log(rowSums(exp(X - max.X))) + max.X
}

# Estimate the log likelihood
# @return norm
# @return resp is a n x k matrix. It indicates the probability
#        that a data point belongs to a cluster.
fm.estep <- function(X, params, cov.type)
{
	weighted.logprob <- est.weighted.logprob(X, params$means,
											 params$covs, cov.type,
											 params$weights)
	logprob.norm <- logsumexp(weighted.logprob)
	log.resp <- weighted.logprob - logprob.norm
	norm <- mean(logprob.norm)
	fm.materialize(log.resp, logprob.norm, norm)
	list(norm=as.vector(norm), resp=log.resp)
}

# This estimate the parameters.
fm.mstep <- function(X, log.resp, reg.covar, cov.type)
{
	params <- esti.gaussian.params(X, exp(log.resp), reg.covar, cov.type)
	list(weights=params$weights, means=params$means, covs=params$covs)
}

compute.lower.bound <- function(log.resp, log.norm)
{
	log.norm
}

# Fit a mixture of Gaussian distribution on the data.
#
# @param X is a n x d matrix. It's the input data.
# @param k the number of components.
# @param reg.covar is a real value. It's added to the diagonal of
#        the covariance matrix to make it non-singular.
# @param cov.type is the type of covariance matrix. It can be
#        one of {"full", "tied", "diag", "spherical"}.
#        \itemize{
#        \item{"full"}{each component has its own general covariance matrix.}
#        \item{"tied"}{all components share the same general covariance matrix.}
#        \item{"diag"}{each component has its own diagonal covariance matrix.}
#        \item{"spherical"}{each component has its own single variance.}
#        }
# @return 
#        \itemize{
#        \item{loglik}{a n x k matrix, whose \code{[i, k]}th entry is
#                the conditional probability of the ith observation
#                belonging to the kth component of the mixture.}
#        \item{iter}{the number of iterations}
#        \item{parameters}{parameters of the mixture of Gaussian distribution.
#             \itemize{
#             \item{weights}{a vector of k elements. Each element is
#              the weight of the Gaussian distribution in the mixture.}
#             \item{means}{a k x d matrix. Each row is the mean of a Gaussian distribution.}
#             \item{covs}{a list of matrices, a matrix or a vector, depending on \code{cov.type}}}
#        }
GMM.fit <- function(X, k, max.iter=100, tol=1e-3, reg.covar=1e-6,
					method=c("random", "random_params", "kmeans"),
					cov.type=c("full", "tied", "diag", "spherical"))
{
	method <- match.arg(method)
	cov.type <- match.arg(cov.type)
	params <- init.params(X, k, reg.covar, method, cov.type)
	for (i in 1:max.iter) {
		eret <- fm.estep(X, params, cov.type)
		params <- fm.mstep(X, eret$resp, reg.covar, cov.type)
		lb <- compute.lower.bound(eret$resp, eret$norm)
		if (i > 5 && abs(lb - prev.lb) < tol)
			break
		prev.lb <- lb
		gc()
	}
	structure(list(loglik=eret$resp, score=eret$norm, iter=i,
				   cov.type=cov.type, parameters=params), class="GMM")
}


In [20]:
start <- Sys.time()
res <- GMM.fit(data1, 10, max.iter = 5, method = "random_params")
end <- Sys.time()
print(end - start)

Time difference of 20.30929 mins


# Measure the performance of FlashR on SSDs

In [16]:
emdata <- fm.conv.store(data, in.mem = FALSE)
emy <- fm.conv.store(y, in.mem = FALSE)

In [12]:
#for (i in 1:5) {
    start <- Sys.time()
    res <- cor(emdata)
    end <- Sys.time()
    print(end - start)
#}

Time difference of 11.23371 secs


In [13]:
#for (i in 1:5) {
    start <- Sys.time()
    res <- fm.svd(emdata, nu=0, nv=ncol(data))
    end <- Sys.time()
    print(end - start)
#}

Time difference of 8.299418 secs


In [17]:
start <- Sys.time()
res <- naiveBayes.train(emdata, emy)
end <- Sys.time()
print(end - start)

Time difference of 6.054668 secs


In [14]:
emdata1 <- fm.conv.store(data1, in.mem = FALSE)

In [15]:
start <- Sys.time()
res <- fm.kmeans(emdata1, 10, iter.max = 5, use.blas = TRUE)
end <- Sys.time()
print(end - start)

KMeans takes 5 iterations and 59.61842 seconds
Time difference of 1.182798 mins


In [21]:
start <- Sys.time()
res <- GMM.fit(emdata1, 10, max.iter = 5, method = "random_params")
end <- Sys.time()
print(end - start)

Time difference of 20.80116 mins


# Measure the performance of logistic regression

This requires to use a real-world dataset.

In [2]:
data <- fm.read.obj("/mnt/data/Criteo_2days.mat")

In [4]:
start <- Sys.time()
res <- logistic.regression(data[,2:40], data[,1], method="L-BFGS-B", max.iters=14)
end <- Sys.time()
print(end - start)

[1] "cost=0.693147180559947, grad=10843505.5358365"
[1] "cost=764003.400438973, grad=765598.641434945"
[1] "cost=230656.518893013, grad=765598.641432455"
[1] "cost=69636.3965749515, grad=765598.641013303"
[1] "cost=21023.6071714191, grad=765598.61732509"
[1] "cost=6347.16156844917, grad=765598.382012076"
[1] "cost=1916.27325607795, grad=765596.063092687"
[1] "cost=578.573503513273, grad=765590.646832222"
[1] "cost=174.71843061944, grad=765587.609972131"
[1] "cost=52.7929091937702, grad=765586.689310092"
[1] "cost=15.982987309764, grad=765586.422706491"
[1] "cost=4.86995906609609, grad=765586.337194358"
[1] "cost=1.51517971894398, grad=765584.98068753"
[1] "cost=0.503526057994861, grad=764573.682139315"
Iteration 1: 
fx = 0.503526

  xnorm = 6.22987e-07, gnorm = 764574, step = 5.74525e-14

[1] "cost=0.472174545891044, grad=764084.806030295"
[1] "cost=0.3472624872472, grad=755794.086766267"
[1] "cost=5.35974636061091, grad=22247550.098131"
[1] "cost=0.30774941233625, grad=745597.01521050

In [5]:
emdata <- fm.conv.store(data, in.mem=FALSE)

In [6]:
start <- Sys.time()
res <- logistic.regression(emdata[,2:40], emdata[,1], method="L-BFGS-B", max.iters=14)
end <- Sys.time()
print(end - start)

[1] "cost=0.693147180559947, grad=10843505.5358365"
[1] "cost=764003.400438973, grad=765598.641434945"
[1] "cost=230656.518893013, grad=765598.641432455"
[1] "cost=69636.3965749515, grad=765598.641013303"
[1] "cost=21023.6071714191, grad=765598.61732509"
[1] "cost=6347.16156844917, grad=765598.382012076"
[1] "cost=1916.27325607795, grad=765596.063092687"
[1] "cost=578.573503513273, grad=765590.646832222"
[1] "cost=174.71843061944, grad=765587.609972131"
[1] "cost=52.7929091937702, grad=765586.689310092"
[1] "cost=15.982987309764, grad=765586.422706491"
[1] "cost=4.86995906609609, grad=765586.337194358"
[1] "cost=1.51517971894398, grad=765584.98068753"
[1] "cost=0.503526057994861, grad=764573.682139315"
Iteration 1: 
fx = 0.503526

  xnorm = 6.22987e-07, gnorm = 764574, step = 5.74525e-14

[1] "cost=0.472174545891044, grad=764084.806030295"
[1] "cost=0.3472624872472, grad=755794.086766267"
[1] "cost=5.35974636061092, grad=22247550.098131"
[1] "cost=0.30774941233625, grad=745597.01521050